In [1]:
from src.domain.model import Model
from src.domain.datamodels import DatasetConfig, ModelConfig
from pathlib import Path
from src.utils.helper_funtions import load_model_config
from src.domain.dataset.bridge_dataset import BridgeDataset

# First, let's set up our model and generate some outputs as shown previously
dataset_path = Path("input_data.pkl")
checkpoint_path = Path("model_artifacts/2025_02_09_run_001/model_epoch_5.pth")

config_dict = load_model_config(checkpoint_path)
dataset_config = DatasetConfig(**config_dict["dataset_config"])
model_config = ModelConfig(**config_dict["model_config"])

# Initialize dataset and model
dataset = BridgeDataset(dataset_config)
model = Model(model_config, dataset_config)

# Load weights and set model to evaluation mode
model.load_state_dict(config_dict["model_state_dict"])
model.eval()

# Generate outputs for a few example words
words = ["cat", "hat"]
encodings = dataset.encode(words)

/home/uzair/Workspace/BRIDGE/src/utils/helper_funtions.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


In [7]:
output = model.generate(encodings, pathway="o2p").model_dump()

In [8]:
decoded_results = []
phon_tokens = output.get("phon_tokens")
phon_token_list = phon_tokens.tolist() if hasattr(phon_tokens, "tolist") else phon_tokens

for i, token_seq in enumerate(phon_token_list):
    decoded_word = dataset.phonemizer.decode(token_seq)
    if i < len(words):
        decoded_results.append({words[i]: decoded_word})
    else:
        decoded_results.append(decoded_word)

No phoneme found for feature vector: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0). Calculating cosine similarity.
No phoneme found for feature vector: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0). Calculating cosine similarity.
No phoneme found for feature vector: (0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0). Calculating cosine similarity.
No phoneme found for feature vector: (0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0). Calculating cosine similarity.
No phoneme found for feature vector: (0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0). Calculating cosine similarity.
No phoneme found for feature vector: (0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0). Calculating cosine similarity.
No phoneme found for f

In [9]:
decoded_results

[{'cat': ['AX0', 'IH1', 'M', 'CH', 'AH0', 'K', 'IX0']},
 {'hat': ['Q', 'IH1', 'M', 'CH', 'M', 'AY0']}]